In [297]:
#import libraires
import requests
from bs4 import BeautifulSoup
import pandas as pd

url='https://frontendmasters.com/courses/'
base_url='https://frontendmasters.com'

In [298]:
def get_doc(url):
  response=requests.get(url)
  if(response.status_code!=200):
    print(f"unable to load the page: {url}")
  doc=BeautifulSoup(response.text)
  return doc

In [299]:
# we will scrap course name, course instructor, isntructor working at

def get_course_tags(url):
  doc=get_doc(url)
  course_title_tags=doc.find_all('h2',class_='title')
  course_inst_tags=doc.find_all('div',class_='name')
  inst_org_tags=doc.find_all('div',{'class':'organization'})
  course_link_tags=doc.find_all('h2',class_='title')

  course_dict={
      "title":[],
      "instructor":[],
      "organization":[],
      "link":[]
  }
  for i in range(len(course_title_tags)):
    course_dict['title'].append(course_title_tags[i].text)
    course_dict['instructor'].append(course_inst_tags[i].text)
    course_dict['organization'].append(inst_org_tags[i].text)
    course_dict['link'].append(base_url+course_link_tags[i].find('a')['href'])
  return pd.DataFrame(course_dict)

In [300]:
course_df=get_course_tags(url)
# %load_ext google.colab.data_table
course_df
# course_df['link']
# course_df.to_csv('course.csv',index=None)

,title,instructor,organization,link
0,Dynamic CSS with Custom Properties (aka CSS Va...,Lea Verou,MIT,https://frontendmasters.com/courses/css-variab...
1,Intermediate React Native,Kadi Kraman,Formidable,https://frontendmasters.com/courses/intermedia...
2,"CSS Grid & Flexbox for Responsive Layouts, v2",Jen Kramer,Freelance Instructor,https://frontendmasters.com/courses/css-grid-f...
3,Developer Productivity,ThePrimeagen,Netflix,https://frontendmasters.com/courses/developer-...
4,Complete Intro to Real-Time,Brian Holt,Stripe,https://frontendmasters.com/courses/realtime/
...,...,...,...,...
128,The Good Parts of JavaScript and the Web,Douglas Crockford,Paypal,https://frontendmasters.com/courses/good-parts...
129,Rethinking Asynchronous JavaScript,Kyle Simpson,You Don't Know JS,https://frontendmasters.com/courses/rethinking...
130,Asynchronous Programming in JavaScript (with R...,Jafar Husain,Netflix,https://frontendmasters.com/courses/asynchrono...
131,Organizing JavaScript Functionality,Kyle Simpson,You Don't Know JS,https://frontendmasters.com/courses/organizing...


In [301]:
def get_topic(url):
  get_doc(url)
  topic_tag=doc.find_all('h3')
  link_tag=doc.find_all('li',class_='LessonListItem')
  dur_tag=doc.find_all('div',class_='timestamp')
  details_dict={
      'topic':[],
      'link':[],
      'duration':[]
  }
  for i in range(len(topic_tag)):
    details_dict['topic'].append(topic_tag[i].text)
    details_dict['link'].append(base_url+link_tag[i].find('a')['href'])
    details_dict['duration'].append(get_time_difference(dur_tag[i].text.split('-')))
  topics_df=pd.DataFrame(details_dict)
  return topics_df
  # return details_dict

get_topic('https://frontendmasters.com/courses/css-variables/')

,topic,link,duration
0,Introduction,https://frontendmasters.com/courses/css-variab...,3 mins 25 seconds
1,CSS Variables Overview,https://frontendmasters.com/courses/css-variab...,8 mins 2 seconds
2,Inheritance,https://frontendmasters.com/courses/css-variab...,11 mins 23 seconds
3,Creating Style Hooks,https://frontendmasters.com/courses/css-variab...,6 mins 83 seconds
4,Fallbacks,https://frontendmasters.com/courses/css-variab...,13 mins 98 seconds
5,Fallbacks Exercise,https://frontendmasters.com/courses/css-variab...,0 mins 55 seconds
6,Fallbacks Solution,https://frontendmasters.com/courses/css-variab...,2 mins 92 seconds
7,DRY Fallback Strategies,https://frontendmasters.com/courses/css-variab...,11 mins 2 seconds
8,Invalid Values & Computed Values,https://frontendmasters.com/courses/css-variab...,12 mins 32 seconds
9,@supports & Custom Properties,https://frontendmasters.com/courses/css-variab...,2 mins 63 seconds


In [302]:
def get_time(t):
  tl=t.split(':')
  for i in range(len(tl)):
      if(i==0):
        tl[i]=int(tl[i])*3600
      elif(i==1):
        tl[i]=int(tl[i])*60
      else:
        tl[i]=int(tl[i])
  seconds=tl[0]+tl[1]+tl[2]
  return round(seconds,2)

# get_time('00:02:15')

[]

def get_time_difference(time):
  time=round((get_time(time[1])-get_time(time[0]))/60,2)
  list=str(time).split('.')
  return f"{list[0]} mins {list[1]} seconds"
get_time_difference(['00:00:00','00:03:15'])  

'3 mins 25 seconds'

In [304]:
#now we will create course wise table of content and its duration

def course_detail(url):
  course_df=get_course_tags(url)
  for i in range(len(course_df['link'])):
    url= course_df['link'][i]
    get_topic(url).to_csv(course_df['title'][i][0]+".csv",index='None')

course_detail(url)
  

In [296]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


'3 mins 25 seconds'